# رزرو هتل با میان‌افزار عضو اولویت‌دار

این نوت‌بوک **میان‌افزار مبتنی بر تابع** را با استفاده از Microsoft Agent Framework نشان می‌دهد. ما بر روی مثال گردش کار شرطی می‌سازیم و یک لایه میان‌افزار اضافه می‌کنیم که به اعضای اولویت‌دار امتیازات ویژه می‌دهد.

## آنچه یاد خواهید گرفت:
1. **میان‌افزار مبتنی بر تابع**: رهگیری و اصلاح نتایج تابع
2. **دسترسی به زمینه**: خواندن و اصلاح `context.result` پس از اجرا
3. **اجرای منطق کسب‌وکار**: مزایای اعضای اولویت‌دار
4. **بازنویسی نتیجه**: تغییر نتایج تابع بر اساس وضعیت کاربر
5. **همان گردش کار، نتایج متفاوت**: تغییر رفتار بر اساس میان‌افزار

## معماری گردش کار با میان‌افزار:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## تفاوت کلیدی با گردش کار شرطی:

**بدون میان‌افزار** (14-conditional-workflow.ipynb):
- پاریس اتاقی ندارد → ارسال به alternative_agent

**با میان‌افزار** (این نوت‌بوک):
- کاربر عادی + پاریس → اتاق نیست → ارسال به alternative_agent
- کاربر اولویت‌دار + پاریس → 🌟 میان‌افزار بازنویسی کرد! → موجود → ارسال به booking_agent

## پیش‌نیازها:
- نصب Microsoft Agent Framework
- آشنایی با گردش‌های کاری شرطی (نگاه کنید به 14-conditional-workflow.ipynb)
- توکن گیت‌هاب یا کلید API OpenAI
- درک پایه‌ای از الگوهای میان‌افزار


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## گام ۱: تعریف مدل‌های Pydantic برای خروجی‌های ساخت‌یافته

این مدل‌ها **طرح‌واره** را که عوامل بازمی‌گردانند تعریف می‌کنند. ما یک فیلد `priority_override` اضافه کرده‌ایم تا زمان‌هایی که میان‌افزار نتیجه در دسترس بودن را تغییر می‌دهد، ردیابی شود.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## مرحله ۲: تعریف پایگاه داده اعضای اولویت دار

برای این دموی نمایشی، ما یک پایگاه داده عضویت اولویت دار را شبیه‌سازی می‌کنیم. در محیط تولید، این کار کوئری‌گیری از یک پایگاه داده واقعی یا API خواهد بود.

**اعضای اولویت دار:**
- `alice@example.com` - عضو VIP
- `bob@example.com` - عضو پریمیوم  
- `priority_user` - حساب آزمایشی


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## گام ۳: ایجاد ابزار رزرو هتل

همانند جریان کاری شرطی، اما اکنون توسط میان‌افزار رهگیری خواهد شد!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## مرحله ۴: 🌟 ایجاد میان‌افزار بررسی اولویت (ویژگی کلیدی!)

این عملکرد **اصلی** این دفترچه است. میان‌افزار:

۱. **نظارت** بر فراخوانی تابع hotel_booking را انجام می‌دهد  
۲. **تابع را به طور معمول اجرا** می‌کند با فراخوانی `next(context)`  
۳. **نتیجه را در** `context.result` **بازرسی می‌کند**  
۴. **نتیجه را در صورتی که کاربر اولویت داشته باشد و اتاق موجود نباشد، بازنویسی می‌کند**  
۵. **نتیجه تغییر یافته را به عامل برمی‌گرداند**

**الگوی کلیدی:**  
```python
async def my_middleware(context, next):
    await next(context)  # اجرای تابع
    # اکنون context.result شامل خروجی تابع است
    if some_condition:
        context.result = new_value  # بازنویسی!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## مرحله ۵: تعریف توابع شرطی برای مسیر‌یابی

توابع شرطی مشابه با جریان کاری شرطی - آن‌ها خروجی ساختاریافته را بررسی می‌کنند تا مسیر‌یابی را تعیین کنند.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## مرحله ۶: ایجاد اجرای سفارشی نمایش

همان اجرای قبلی - خروجی نهایی گردش کار را نمایش می‌دهد.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## مرحله ۷: بارگذاری متغیرهای محیطی

کلاینت LLM را پیکربندی کنید (مدل‌های GitHub یا OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## مرحله ۸: ایجاد عوامل هوش مصنوعی با میان‌افزار

**تفاوت اصلی:** هنگام ایجاد availability_agent، پارامتر `middleware` را می‌فرستیم!

این‌گونه است که middleware بررسی اولویت را در خط لوله‌ی فراخوانی عملکرد عامل تزریق می‌کنیم.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## مرحله ۹: ساخت گردش کار

ساختار گردش کار مشابه قبل – مسیر یابی شرطی بر اساس در دسترس بودن.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## مرحله ۱۰: مورد آزمایشی ۱ - کاربر عادی در پاریس (بدون لغو دستی)

یک کاربر عادی تلاش می‌کند پاریس را رزرو کند → هیچ اتاقی وجود ندارد → مسیر به alternative_agent هدایت می‌شود


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## مرحله ۱۱: مورد آزمایشی ۲ - 🌟 کاربر اولویت‌دار در پاریس (با بازنویسی!)

یک عضو اولویت‌دار سعی می‌کند پاریس را رزرو کند → در ابتدا هیچ اتاقی وجود ندارد → 🌟 میدلور بازنویسی می‌کند! → مسیریابی به booking_agent

**این نمایش کلیدی قدرت میدلور است!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## گام ۱۲: مورد آزمایشی ۳ - کاربر اولویت‌دار در استکهلم (قبلاً موجود)

کاربر اولویت‌دار سعی می‌کند استکهلم → اتاق‌ها در دسترس هستند → نیازی به تغییر وجود ندارد → مسیر به booking_agent هدایت می‌شود

این نشان می‌دهد که میان‌افزار فقط زمانی عمل می‌کند که لازم باشد!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## نکات کلیدی و مفاهیم میان‌افزار

### ✅ مواردی که آموختید:

#### **1. الگوی میان‌افزار مبتنی بر تابع**

میان‌افزار تماس‌های تابع را با استفاده از یک تابع ساده async رهگیری می‌کند:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # قبل از اجرای تابع
    print("Intercepting...")
    
    # تابع را اجرا کنید
    await next(context)
    
    # پس از اجرای تابع - نتیجه را بررسی کنید
    if context.result:
        # در صورت نیاز نتیجه را تغییر دهید
        context.result = modified_value
```

#### **2. دسترسی به context و بازنویسی نتیجه**

- `context.function` - دسترسی به تابعی که فراخوانی شده است  
- `context.arguments` - خواندن آرگومان‌های تابع  
- `context.kwargs` - دسترسی به پارامترهای اضافی  
- `await next(context)` - اجرای تابع  
- `context.result` - خواندن/ویرایش خروجی تابع  

#### **3. پیاده‌سازی منطق تجاری**

میان‌افزار ما قابلیت‌های اعضای اولویت‌دار را پیاده‌سازی می‌کند:  
- **کاربران عادی**: بدون تغییر، روند کاری استاندارد  
- **کاربران اولویت‌دار**: بازنویسی "عدم دسترسی" → "دسترسی موجود"  
- **منطق شرطی**: فقط در مواقع نیاز بازنویسی می‌کند  

#### **4. همان روند کاری، نتایج متفاوت**

قدرت میان‌افزار:  
- ✅ بدون تغییر در ساختار روند کاری  
- ✅ بدون تغییر در تابع ابزار  
- ✅ بدون تغییر در منطق شرطی مسیریابی  
- ✅ فقط میان‌افزار → رفتار متفاوت!

### 🚀 کاربردهای واقعی:

1. **ویژگی‌های VIP/پریمیوم**  
   - بازنویسی محدودیت‌های نرخ برای کاربران پریمیوم  
   - ارائه دسترسی اولویت‌دار به منابع  
   - فعال‌سازی پویا ویژگی‌های پریمیوم  

2. **آزمون A/B**  
   - هدایت کاربران به پیاده‌سازی‌های متفاوت  
   - آزمایش ویژگی‌های جدید با کاربران مشخص  
   - عرضه تدریجی ویژگی‌ها  

3. **امنیت و تطابق**  
   - بررسی و ممیزی فراخوانی‌های تابع  
   - مسدودسازی عملیات حساس  
   - اجرای قوانین کسب‌وکار  

4. **بهینه‌سازی عملکرد**  
   - کش کردن نتایج برای کاربران خاص  
   - پرش از عملیات پرهزینه در صورت امکان  
   - تخصیص پویا منابع  

5. **مدیریت خطا و تلاش مجدد**  
   - گرفتن خطاها و مدیریت آن‌ها به صورت نرم  
   - پیاده‌سازی منطق تلاش مجدد  
   - بازگشت به پیاده‌سازی‌های جایگزین  

6. **لاگ‌گیری و مانیتورینگ**  
   - ثبت زمان اجرای توابع  
   - لاگ‌گیری پارامترها و نتایج  
   - پایش الگوهای استفاده  

### 🔑 تفاوت‌های کلیدی نسبت به دکوراتورها:

| ویژگی | دکوراتور | میان‌افزار |
|---------|-----------|------------|
| **دامنه** | یک تابع | همه توابع در عامل |
| **انعطاف‌پذیری** | ثابت در تعریف | پویا در زمان اجرا |
| **context** | محدود | context کامل عامل |
| **ترکیب‌پذیری** | چند دکوراتور | خط لوله میان‌افزار |
| **آگاهی از عامل** | خیر | بله (دسترسی به وضعیت عامل) |

### 📚 زمان استفاده از میان‌افزار:

✅ **میان‌افزار را زمانی استفاده کنید که:**  
- نیاز به تغییر رفتار بر اساس وضعیت کاربر/جلسه دارید  
- می‌خواهید منطق را روی چندین تابع اعمال کنید  
- نیاز به دسترسی به context سطح عامل دارید  
- در حال پیاده‌سازی مواردی با نگرانی‌های مشترک (لاگ‌گیری، احراز هویت، و غیره) هستید  

❌ **میان‌افزار را زمانی استفاده نکنید که:**  
- اعتبارسنجی ساده ورودی (از Pydantic استفاده کنید)  
- منطق خاص تابع (در خود تابع بماند)  
- تغییرات یک‌باره (کافی است تابع را تغییر دهید)  

### 🎓 الگوهای پیشرفته:

```python
# چندین میدل‌ویر (ترتیب اجرا مهم است!)
middleware=[
    logging_middleware,      # ابتدا لاگ‌ها
    auth_middleware,         # سپس بررسی احراز هویت
    cache_middleware,        # سپس بررسی کش
    rate_limit_middleware,   # سپس محدودیت نرخ
    priority_check_middleware  # در نهایت بررسی اولویت
]

# اجرای میدل‌ویر شرطی
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # اصلاح نتیجه
    else:
        # کاملاً اجرای را رد کن
        context.result = cached_value
```

### 🔗 مفاهیم مرتبط:

- **میان‌افزار عامل**: رهگیری تماس agent.run()  
- **میان‌افزار تابع**: رهگیری تماس‌های تابع ابزار (همان که استفاده کردیم!)  
- **خط لوله میان‌افزار**: زنجیره‌ای از میان‌افزارها که به ترتیب اجرا می‌شوند  
- **انتشار context**: انتقال وضعیت از طریق زنجیره میان‌افزار


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما در تلاش برای دقت هستیم، لطفاً توجه داشته باشید که ترجمه‌های ماشینی ممکن است حاوی خطاها یا نواقصی باشند. سند اصلی به زبان مادری آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما در قبال هر گونه سوءتفاهم یا برداشت نادرست ناشی از استفاده از این ترجمه مسئولیتی نداریم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
